In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add, uint
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

@si.coroutine(inputs={"wdata": si.Bits(4), "wen": si.Bit, "ren": si.Bit})
def Fifo():
    memory = list(bits(0, 4) for i in range(4))
    raddr = uint(0, 2)
    waddr = uint(0, 2)
    prev_empty = True
    prev_full  = False
    wdata, wen, ren = yield
    while True:
        full = prev_full
        empty = prev_empty
        if wen and not prev_full:
            memory[waddr] = wdata
            waddr = waddr + 1
            full = raddr == waddr
            empty = False
        rdata = memory[raddr]
        if ren and not prev_empty:
            raddr = raddr + 1
            empty = raddr == waddr
            full = False
        prev_full = full
        prev_empty = empty
        wdata, wen, ren = yield rdata, empty, full


        
si.compile(Fifo(), file_name="build/silica_fifo.py")
with open("build/silica_fifo.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width, strategy):
    if strategy == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
               

In [2]:
!magma -o coreir -m coreir -t Fifo build/silica_fifo.py
!coreir -i build/silica_fifo.json -o build/silica_fifo.v
# !magma -o verilog -m lattice -t Fifo build/silica_fifo.py
!yosys -p 'synth_ice40 -top Fifo -blif build/silica_fifo.blif' build/silica_fifo.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_fifo.txt build/silica_fifo.blif
!icetime -tmd hx1k build/silica_fifo.txt  | grep -B 2 "Total path delay"

In Run Generators

Modified?: Yes
2.27. Printing statistics.

=== Fifo ===

   Number of wires:               2286
   Number of wire bits:           3638
   Number of public wires:        2237
   Number of public wire bits:    3589
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 98
     SB_DFF                         24
     SB_LUT4                        74

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_fifo.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          13 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          77 / 1280
  DFF        24
  CARRY      0
  CARRY, DFF 0
  DFF PASS   3
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted CLK$2, 24 / 24


In [3]:
!yosys -p 'synth_ice40 -top fifo -blif build/verilog_fifo.blif' ../verilog/fifo.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_fifo.txt build/verilog_fifo.blif
!icetime -tmd hx1k build/verilog_fifo.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== fifo ===

   Number of wires:                 26
   Number of wire bits:            107
   Number of public wires:          12
   Number of public wire bits:      69
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 68
     SB_CARRY                       10
     SB_DFFE                        30
     SB_LUT4                        28
seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/verilog_fifo.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          37 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          48 / 1280
  DFF        22
  CARRY      4
  CARRY, DFF 8
  DFF PASS   16
  CARRY PASS 2
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted clk$2, 30